In [21]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime

from dotenv import dotenv_values

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account

from dotenv import dotenv_values
# https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries
# https://codelabs.developers.google.com/codelabs/cloud-bigquery-python#0

# https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# https://cloud.google.com/python/docs/setup


# https://cloud.google.com/apis/docs/getting-started#enabling_apis



# https://cloud.google.com/bigquery/docs/reference/libraries
# https://cloud.google.com/python/docs/reference/bigquery/latest
# https://gcloud.readthedocs.io/en/latest/bigquery-client.html




In [22]:
# credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\xxxxxx.json')
# print(credentials.project_id)

In [23]:
projectId='pongthorn'
dataset_id='SMartDW'
table_id = f"{projectId}.{dataset_id}.incident"
start_date_query='2023-06-01'


env_path='.env'
config = dotenv_values(dotenv_path=env_path)

In [25]:
# client = bigquery.Client(credentials= credentials,project=projectId)
client = bigquery.Client(project=projectId)

In [26]:
try:
    client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
except NotFound:
    raise Exception("Table {} is not found.".format(table_id))

Table pongthorn.SMartDW.incident already exists.


In [27]:
dt_imported=datetime.now()
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Imported DateTime: {str_imported}" )

Imported DateTime: 2023-07-04 14:16:01


In [28]:
sql_lastImport=f"SELECT max(imported_at) as last_imported from `{table_id}` where open_datetime>='{start_date_query}' "
job_lastImported=client.query(sql_lastImport)
str_lastImported=None
for row in job_lastImported:    
    if row.last_imported is not None: 
        str_lastImported=row.last_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Last Imported DateTime: {str_lastImported}" )

if str_lastImported is not None:
  start_date_query=str_lastImported

print(f"Start Import on update_at of last imported date : {start_date_query}" )

Last Imported DateTime: None
Start Import on update_at of last imported date : 2023-06-01


In [29]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [30]:
# check schema 
sql_incident="""

select
incident.id as id, incident.incident_no as incident_no,

severity.id as  severity_id,
severity.severity_name as  severity_name,

service_level.sla_name as sla,

product_type.productype_name as product_type,brand.brand_name as brand,

xtype.incident_type_name as incident_type,
status.incident_status_name as status,
service.service_type_name service_type,

model.model_name as model,CASE WHEN failure_type IS NULL THEN  0 ELSE 1 END AS is_failure_type,


(select count(*) from  app_incident_detail  as detail where  detail.incident_master_id=incident.id ) as count_detail


,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as close_datetime

,TO_CHAR(incident.incident_problem_start  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as response_datetime
,TO_CHAR(incident.incident_problem_end  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as resolved_datetime

,company.company_name as company
,TO_CHAR(incident.updated_at,'YYYY-MM-DD HH24:MI:SS') as updated_at 


from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id
inner join  app_incident_status as status on incident.incident_status_id = status.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join  app_service_type as service on incident.service_type_id= service.id

inner join app_inventory as inventory on incident.inventory_id = inventory.id

inner join app_brand as brand on inventory.brand_id = brand.id
inner join app_model as model on inventory.model_id = model.id
inner join app_product_type as product_type on inventory.product_type_id = product_type.id
inner join app_sla as service_level on inventory.customer_sla_id = service_level.id

inner join app_project as project on inventory.project_id = project.id
inner join app_company as company on project.company_id = company.id

where incident.incident_status_id =4
and incident.updated_at>=%(start_date_param)s

order by incident.incident_datetime desc


"""


In [31]:
print("Create all issues dataframe")

dict_params={"start_date_param":start_date_query}
df_all=list_data(sql_incident,dict_params,get_postgres_conn())

if df_all.empty==True:
    print("no transsaction update")
    exit()


Create all issues dataframe


In [32]:
# convert object to datetime
dateTimeCols=['open_datetime','response_datetime','resolved_datetime','close_datetime','updated_at']
for col in dateTimeCols:
 df_all[col]=pd.to_datetime(df_all[col], format='%Y-%m-%d %H:%M',errors= 'coerce')

df_all['imported_at']=dt_imported


df_all.dropna(inplace=True)

#df_all=df_all.head(10)
print(df_all.info())
df_all.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 179 non-null    int64         
 1   incident_no        179 non-null    object        
 2   severity_id        179 non-null    int64         
 3   severity_name      179 non-null    object        
 4   sla                179 non-null    object        
 5   product_type       179 non-null    object        
 6   brand              179 non-null    object        
 7   incident_type      179 non-null    object        
 8   status             179 non-null    object        
 9   service_type       179 non-null    object        
 10  model              179 non-null    object        
 11  is_failure_type    179 non-null    int64         
 12  count_detail       179 non-null    int64         
 13  open_datetime      179 non-null    datetime64[ns]
 14  close_date

,id,incident_no,severity_id,severity_name,sla,product_type,brand,incident_type,status,service_type,model,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company,updated_at,imported_at
0,3040,SR-ES-23-3040,2,Major,24x7 4Hrs Resolution Time,Firewall,Palo Alto,General Incident,Closed,Incident,PA-5250,0,1,2023-07-04 07:07:00,2023-07-04 08:00:00,2023-07-04 07:08:00,2023-07-04 08:00:00,New-GFMIS,2023-07-04 10:24:27,2023-07-04 14:16:01.174831
1,3036,SR-ES-23-3036,3,Minor,24x7 4Hrs Response Time,Storage,NetApp,OS / Firmware,Closed,Incident,SG5660,0,1,2023-07-03 15:55:00,2023-07-03 17:50:00,2023-07-03 15:55:00,2023-07-03 17:50:00,INET-(NetApp and Non HPE),2023-07-04 09:40:46,2023-07-04 14:16:01.174831
2,3027,SR-ES-23-3027,2,Major,24x7 4Hrs Response Time,Server,HPE,Hard Disk Drive Failure,Closed,Incident,Synergy 660 Gen10,0,2,2023-07-02 04:30:00,2023-07-02 10:00:00,2023-07-02 04:45:00,2023-07-02 10:00:00,TLI,2023-07-03 09:50:50,2023-07-04 14:16:01.174831
3,3042,SR-ES-23-3042,2,Major,24x7 4Hrs Resolution Time,Server,HPE,General Incident,Closed,Incident,StoreOnce 5250,0,1,2023-07-01 08:30:00,2023-07-01 15:00:00,2023-07-01 08:30:00,2023-07-01 15:00:00,New-GFMIS,2023-07-04 10:53:11,2023-07-04 14:16:01.174831
4,3039,SR-ES-23-3039,2,Major,24x7 4Hrs Resolution Time,Firewall,Palo Alto,General Incident,Closed,Incident,PA-5250,0,1,2023-07-01 07:12:00,2023-07-01 08:00:00,2023-07-01 07:12:00,2023-07-01 08:00:00,New-GFMIS,2023-07-04 10:20:54,2023-07-04 14:16:01.174831
5,3029,SR-ES-23-3029,4,Cosmetic,24x7 4Hrs Response Time,Storage,Veritas,General Incident,Closed,Incident,Veritas NETBACKUP APPLIANCE 5240,0,4,2023-06-30 22:47:00,2023-07-03 13:47:00,2023-06-30 23:10:00,2023-07-01 21:00:00,Coforge (NIIT),2023-07-03 17:01:41,2023-07-04 14:16:01.174831
6,3030,SR-ES-23-3030,3,Minor,24x7 4Hrs Response Time,Storage,NetApp,Hard Disk Drive Failure,Closed,Incident,E2824,0,1,2023-06-29 10:30:00,2023-07-03 12:00:00,2023-06-29 10:30:00,2023-07-03 12:00:00,INET-(NetApp and Non HPE),2023-07-03 13:39:44,2023-07-04 14:16:01.174831
7,3031,SR-ES-23-3031,3,Minor,24x7 4Hrs Response Time,Security,Trend Micro,Upgrade Software,Closed,Incident,Deep security,0,2,2023-06-29 10:00:00,2023-07-01 21:00:00,2023-06-29 10:00:00,2023-07-01 21:00:00,INET-(NetApp and Non HPE),2023-07-03 13:51:16,2023-07-04 14:16:01.174831
8,3021,SR-ES-23-3021,3,Minor,24x7 6Hrs Response Time,Server,HPE,General Incident,Closed,Incident,Synergy 480 Gen10,0,1,2023-06-28 18:55:00,2023-06-30 08:58:00,2023-06-28 19:07:00,2023-06-30 08:58:00,INET- (HPE Product),2023-06-30 08:59:48,2023-07-04 14:16:01.174831
9,3035,SR-ES-23-3035,3,Minor,24x7 4Hrs Resolution Time,Storage,NetApp,Controller/Node Failure,Closed,Incident,SG5712,0,6,2023-06-28 15:21:00,2023-06-30 01:36:00,2023-06-28 16:00:00,2023-06-28 16:52:00,AIS,2023-07-03 23:33:21,2023-07-04 14:16:01.174831


In [33]:
def insertDataFrameToBQ(df_trasns):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            df_trasns, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_trasns), "Imported closed incident to bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        print(e) 

try:
    insertDataFrameToBQ(df_all)
except Exception as ex:
    raise ex




Total  179 Imported closed incident to bigquery successfully


In [34]:
importDate=dt_imported.strftime("%Y-%m-%d")

def load_data_bq(sql:str):
 query_result=client.query(sql)
 dfx=query_result.to_dataframe()
 return dfx
sql_query=f""" SELECT * FROM `{table_id}` WHERE imported_at >= '{importDate}' LIMIT 10 """
df=load_data_bq(sql_query)
df

,id,incident_no,severity_id,severity_name,sla,product_type,brand,model,incident_type,status,service_type,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company,updated_at,imported_at
0,2894,SR-ES-23-2894,1,Critical,24x7 4Hrs Resolution Time,Server,Oracle,SPARC T5-2,System Board Failure,Closed,Incident,0,2,2023-05-23 08:08:00,2023-05-24 21:16:00,2023-05-23 09:05:00,2023-05-24 21:16:00,MEA (ฝคฐ.),2023-06-07 21:16:51,2023-07-04 14:16:01.174831
1,2952,SR-ES-23-2952,1,Critical,8x5 4Hrs Response Time,Server,QNAP,QNP-TS-451D2-4G,Controller/Node Failure,Closed,Incident,0,6,2023-06-19 13:16:00,2023-06-22 18:57:00,2023-06-19 13:39:00,2023-06-21 22:50:00,Coforge (NIIT),2023-06-22 19:07:15,2023-07-04 14:16:01.174831
2,2708,SR-ES-23-2708,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,SimpliVity 380 Gen9 Node,Upgrade Software,Closed,Incident,0,8,2023-04-26 06:00:00,2023-06-10 11:00:00,2023-04-26 06:00:00,2023-06-09 18:00:00,KTBCS,2023-06-11 12:52:31,2023-07-04 14:16:01.174831
3,2815,SR-ES-23-2815,1,Critical,24x7 4Hrs Response Time,Storage,Veritas,Veritas NETBACKUP APPLIANCE 5240,Backup Failure,Closed,Incident,0,2,2023-05-23 06:52:00,2023-05-24 15:45:00,2023-05-23 07:12:00,2023-05-23 21:00:00,Coforge (NIIT),2023-06-05 21:18:14,2023-07-04 14:16:01.174831
4,2939,SR-ES-23-2939,2,Major,24x7 6Hrs Resolution Time,Storage,YIP,dummy,General Incident,Closed,Request,0,2,2023-06-14 14:10:00,2023-06-15 01:30:00,2023-06-15 01:30:00,2023-06-15 01:30:00,GLO,2023-06-15 11:15:13,2023-07-04 14:16:01.174831
5,2905,SR-ES-23-2905,2,Major,24x7 6Hrs Resolution Time,Storage,YIP,dummy,Software,Closed,Request,0,2,2023-06-10 17:10:00,2023-06-10 18:30:00,2023-06-10 17:15:00,2023-06-10 18:20:00,GLO,2023-06-12 11:38:05,2023-07-04 14:16:01.174831
6,2892,SR-ES-23-2892,2,Major,24x7 4Hrs Response Time,Storage,NetApp,SG5660,OS / Firmware,Closed,Incident,0,2,2023-05-02 11:30:00,2023-06-07 10:30:00,2023-05-02 11:30:00,2023-06-07 10:30:00,INET-(NetApp and Non HPE),2023-06-07 16:04:55,2023-07-04 14:16:01.174831
7,2976,SR-ES-23-2976,2,Major,24x7 4Hrs Response Time,Storage,NetApp,SG5712,Controller/Node Failure,Closed,Incident,0,3,2023-06-16 09:12:00,2023-06-16 13:03:00,2023-06-16 09:12:00,2023-06-16 09:48:00,AIS,2023-06-26 14:28:39,2023-07-04 14:16:01.174831
8,2929,SR-ES-23-2929,2,Major,24x7 4Hrs Response Time,Storage,NetApp,SG5712,Controller/Node Failure,Closed,Incident,0,6,2023-06-12 21:00:00,2023-06-16 13:01:00,2023-06-12 21:58:00,2023-06-13 10:01:00,AIS,2023-06-26 14:33:01,2023-07-04 14:16:01.174831
9,2927,SR-ES-23-2927,2,Major,24x7 4Hrs Response Time,Storage,NetApp,SG5712,Controller/Node Failure,Closed,Incident,0,2,2023-06-09 02:28:00,2023-06-13 13:53:00,2023-06-09 14:34:00,2023-06-09 19:50:00,AIS,2023-06-26 13:37:14,2023-07-04 14:16:01.174831
